In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/transformers/facebook-bart-large/config.json
/kaggle/input/transformers/facebook-bart-large/tokenizer.json
/kaggle/input/transformers/facebook-bart-large/pytorch_model.bin
/kaggle/input/transformers/bert-large-uncased/config.json
/kaggle/input/transformers/bert-large-uncased/tokenizer.json
/kaggle/input/transformers/bert-large-uncased/pytorch_model.bin
/kaggle/input/transformers/roberta-base/config.json
/kaggle/input/transformers/roberta-base/tokenizer.json
/kaggle/input/transformers/roberta-base/pytorch_model.bin
/kaggle/input/transformers/distilroberta-base/config.json
/kaggle/input/transformers/distilroberta-base/tokenizer.json
/kaggle/input/transformers/distilroberta-base/pytorch_model.bin
/kaggle/input/transformers/distilbert-base-uncased/config.json
/kaggle/input/transformers/distilbert-base-uncased/tokenizer.json
/kaggle/input/transformers/distilbert-base-uncased/pytorch_model.bin
/kaggle/input/transformers/xlnet-base-cased/config.json
/kaggle/input/transformers/xl

In [2]:
from sklearn.preprocessing import LabelEncoder
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import regex as re
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import csv
from nltk.stem import PorterStemmer
import string
from nltk.stem import WordNetLemmatizer
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from transformers import RobertaTokenizer, AutoModel,AutoTokenizer,AutoModelForQuestionAnswering

## constant

In [3]:
train_path     = "/kaggle/input/lmsys-chatbot-arena/train.csv"
test_path      = "/kaggle/input/lmsys-chatbot-arena/test.csv"
model_path     = '../input/transformers/roberta-base'



In [4]:
train = pd.read_csv(train_path)
test  = pd.read_csv(test_path)

In [5]:
train

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0


In [6]:
target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

In [7]:
y = train[target_columns].idxmax(axis=1) 

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [8]:
labels = torch.tensor(y_encoded, dtype=torch.int64)
labels

tensor([0, 1, 2,  ..., 0, 1, 0])

In [9]:
train["all"] = train.apply(lambda train: f"User prompt: {train['prompt']}\n\nModel A :\n{train['response_a']}\n\n--------\n\nModel B:\n{train['response_b']}", axis=1).tolist()


In [10]:
test["all"] =test.apply(lambda test: f"User prompt: {test['prompt']}\n\nModel A :\n{test['response_a']}\n\n--------\n\nModel B:\n{test['response_b']}", axis=1).tolist()

In [11]:
X_train=train["all"].tolist()

In [12]:
X_test=test["all"].tolist()

## Prepare data for Model

### 1-load tokenizer

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [14]:
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt",max_length=512)

In [15]:
test_encodings = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt",max_length=512)

In [16]:
train_encodings

{'input_ids': tensor([[    0, 44518, 14302,  ...,   209,  1797,     2],
        [    0, 44518, 14302,  ...,  9889,     6,     2],
        [    0, 44518, 14302,  ..., 12905, 43797,     2],
        ...,
        [    0, 44518, 14302,  ...,  2139, 48347,     2],
        [    0, 44518, 14302,  ...,     1,     1,     1],
        [    0, 44518, 14302,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

### 2-custom dataset for train

In [17]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [18]:
train_dataset = CustomDataset(train_encodings, labels)

### 2-custom dataset for test

In [19]:
class CustomTestDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [20]:
test_dataset = CustomTestDataset(test_encodings)

### DataLoader


In [21]:
batch_size = 20
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Model

In [22]:
num_classes = 3
num_epochs = 3

In [23]:
model = RobertaForSequenceClassification.from_pretrained(model_path, num_labels=num_classes)



/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ../input/transformers/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
model = model.to("cuda")
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,num_training_steps=len(train_loader) * num_epochs)
     

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
# Replace with your actual number of epochs

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch_idx, batch in enumerate(train_loader):

        # Move data to GPU
        batch = {key: val.to('cuda') for key, val in batch.items()}
            
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        if (batch_idx + 1) % 300 == 0:  
            print(f"Epoch {epoch + 1}/{num_epochs} - Batch {batch_idx + 1}/{len(train_loader)} - Loss: {total_loss / 100:.4f}")
            total_loss = 0.0
        



/tmp/ipykernel_25/136545975.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_25/136545975.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch 1/3 - Batch 300/2874 - Loss: 3.2830
Epoch 1/3 - Batch 600/2874 - Loss: 3.2750
Epoch 1/3 - Batch 900/2874 - Loss: 3.2642
Epoch 1/3 - Batch 1200/2874 - Loss: 3.2710
Epoch 1/3 - Batch 1500/2874 - Loss: 3.2573
Epoch 1/3 - Batch 1800/2874 - Loss: 3.2566
Epoch 1/3 - Batch 2100/2874 - Loss: 3.2492
Epoch 1/3 - Batch 2400/2874 - Loss: 3.2506
Epoch 1/3 - Batch 2700/2874 - Loss: 3.2443
Epoch 2/3 - Batch 300/2874 - Loss: 3.1997
Epoch 2/3 - Batch 600/2874 - Loss: 3.2045
Epoch 2/3 - Batch 900/2874 - Loss: 3.1881
Epoch 2/3 - Batch 1200/2874 - Loss: 3.1610
Epoch 2/3 - Batch 1500/2874 - Loss: 3.1683
Epoch 2/3 - Batch 1800/2874 - Loss: 3.1562
Epoch 2/3 - Batch 2100/2874 - Loss: 3.1669
Epoch 2/3 - Batch 2400/2874 - Loss: 3.1704
Epoch 2/3 - Batch 2700/2874 - Loss: 3.1606
Epoch 3/3 - Batch 300/2874 - Loss: 3.1201
Epoch 3/3 - Batch 600/2874 - Loss: 3.1187
Epoch 3/3 - Batch 900/2874 - Loss: 3.0963
Epoch 3/3 - Batch 1200/2874 - Loss: 3.0582
Epoch 3/3 - Batch 1500/2874 - Loss: 3.1017
Epoch 3/3 - Batch 18

In [26]:
model.eval() 

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [27]:
all_predictions = []

with torch.no_grad():
    for batch in test_loader:
        # Ensure batch is a dictionary
        if isinstance(batch, dict):
            # Move batch to GPU if available
            batch = {key: val.to('cuda') for key, val in batch.items()}

            # Make predictions
            outputs = model(**batch)
            logits = outputs.logits

            # Apply softmax to get probabilities
            probabilities = torch.softmax(logits, dim=-1)

            # Append probabilities to the list
            all_predictions.append(probabilities.cpu().numpy())
        else:
            print(f"Encountered a non-dict batch: {batch}")

# Concatenate all predictions
all_predictions = np.concatenate(all_predictions, axis=0)

# Prepare DataFrame with predictions
submission = pd.DataFrame(all_predictions, columns=['response_b_len', 'prompt_len', 'winner'])


/tmp/ipykernel_25/709883472.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [28]:
submission

,response_b_len,prompt_len,winner
0,0.266004,0.288333,0.445663
1,0.484899,0.216474,0.298627
2,0.360647,0.332232,0.307121


In [29]:
submission.to_csv('submission.csv', index=False)